In [1]:
import sys
import os

from transformers import AutoTokenizer
import datasets
from functools import partial

# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.utils.dataset_tokenization import process_data

In [2]:
# model_path = "../../self-corrective-llama_untrained"
model_name = "MathBite/self_corrective_llama_3.1_8B_untrained"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)

# print(model)

In [3]:
SPECIAL_INSTRUCTION = "\nAs you write your answer, you can correct yourself using these tools: Use <DEL_W> to take back the word before this token, <DEL_S> to remove the entire sentence before this token, and <DEL_A> to scrap everything you've written and start again."
INSERTION_MARKER = "<|start_header_id|>user<|end_header_id|>"
DELETION_MARKERS = ["<DEL_W>", "<DEL_S>", "<DEL_A>"]
DELETION_TOKEN_IDS = tokenizer.convert_tokens_to_ids(DELETION_MARKERS)

In [4]:
# data_path = "../../dataset/train.json"
# dataset = datasets.load_dataset("json", data_files=data_path)

In [5]:
# dataset

In [6]:
# sample = dataset["train"][7295]
# print(sample)
# print(sample["correct_response"])
# print(sample["errors"])
# print(sample["hallucinated_text"])

In [7]:
# for i in range(7295, 7296):
#     sample = dataset["train"][i]
    
#     res = process_data(sample, tokenizer, SPECIAL_INSTRUCTION, INSERTION_MARKER, DELETION_TOKEN_IDS[0], DELETION_TOKEN_IDS[1], DELETION_TOKEN_IDS[2])

#     hall_text_idx = [i for i, label in enumerate(res["hallucination_labels"]) if label == 1]
#     hall_text = [res["input_ids"][i] for i in hall_text_idx]
    
#     print(sample["correct_response"])
#     print("================================================")
#     for error in sample["hallucinated_text"]:
#         print(error)
#     print("================================================")
#     print(tokenizer.decode(hall_text))
#     print("--------------------------------\n\n")

In [8]:
# SPECIAL_INSTRUCTION = "\nAs you write your answer, you can correct yourself using these tools: Use <DEL_W> to take back the word before this token, <DEL_S> to remove the entire sentence before this token, and <DEL_A> to scrap everything you've written and start again."
# INSERTION_MARKER = "<|start_header_id|>user<|end_header_id|>"
# DELETION_MARKERS = ["<DEL_W>", "<DEL_S>", "<DEL_A>"]
# DELETION_TOKEN_IDS = tokenizer.convert_tokens_to_ids(DELETION_MARKERS)

# mapper = partial(
#     process_data,
#     tokenizer=tokenizer,
#     special_instruction=SPECIAL_INSTRUCTION,
#     insertion_marker=INSERTION_MARKER,
#     del_w_token_id=DELETION_TOKEN_IDS[0],
#     del_s_token_id=DELETION_TOKEN_IDS[1],
#     del_a_token_id=DELETION_TOKEN_IDS[2]
# )

In [9]:
# tokenized_dataset = dataset.map(mapper, batched=False)
# tokenized_dataset = tokenized_dataset["train"]
# columns_to_remove = [
#     "input", "correct_response", "incorrect_response", 
#     "additional_info", "errors", "hallucinated_text"
# ]

# tokenized_dataset = tokenized_dataset.remove_columns(columns_to_remove)


In [10]:
# tokenized_dataset

In [11]:
# split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
# print(split_dataset)
# train_dataset = split_dataset['train']
# eval_dataset = split_dataset['test']

In [12]:
# output_dir = "../../dataset/training"
# split_dataset.save_to_disk(output_dir)

In [13]:
dataset = datasets.load_from_disk("../../dataset/training")
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

print(train_dataset)
print(eval_dataset)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels', 'hallucination_labels'],
    num_rows: 36684
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels', 'hallucination_labels'],
    num_rows: 4077
})


In [14]:
for i in range(100):
    sample = train_dataset[i]

    hall_text_idx = [i for i, label in enumerate(sample["hallucination_labels"]) if label == 1]
    hall_text = [sample["input_ids"][i] for i in hall_text_idx]

    print(tokenizer.decode(sample["input_ids"]))
    print("================================================")
    print(tokenizer.decode(hall_text))
    print("--------------------------------\n\n")

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a specialized question-answering AI. Your task is to give a concise answer to the question using *only* the provided context. Make sure to always give an answer.<|eot_id|>
As you write your answer, you can correct yourself using these tools: Use <DEL_W> to take back the word before this token, <DEL_S> to remove the entire sentence before this token, and <DEL_A> to scrap everything you've written and start again.<|start_header_id|>user<|end_header_id|>

Context:
'''
Gaddafi organized demonstrations and distributed posters criticizing the monarchy. In October 1961, he led a demonstration protesting Syria's secession from the United Arab Republic. During this they broke windows of a local hotel accused of serving alcohol. Catching the authorities' attention, they expelled his family from Sabha. Gaddafi moved to Misrata, there attending Misrata Secondary School. Maintaining his interest in Arab nationalist activism, he re

In [15]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    BitsAndBytesConfig,
)
import torch
from src.trainer import SelfCorrectionTrainer

In [16]:
trainer = SelfCorrectionTrainer(model = "None")

In [27]:
sample = train_dataset[0]
sample["input_ids"] = torch.tensor(sample["input_ids"]).reshape(1, -1)
sample["hallucination_labels"] = torch.tensor(sample["hallucination_labels"]).reshape(1, -1)
sample["labels"] = torch.tensor(sample["labels"]).reshape(1, -1)
print(sample)

{'input_ids': tensor([[128000, 128006,   9125, 128007,    271,   2675,    527,    264,  28175,
           3488,     12,    598,     86,   4776,  15592,     13,   4718,   3465,
            374,    311,   3041,    264,  64694,   4320,    311,    279,   3488,
           1701,    353,   3323,      9,    279,   3984,   2317,     13,   7557,
           2771,    311,   2744,   3041,    459,   4320,     13, 128009,    198,
           2170,    499,   3350,    701,   4320,     11,    499,    649,   4495,
           6261,   1701,   1521,   7526,     25,   5560,    220, 128256,    311,
           1935,   1203,    279,   3492,   1603,    420,   4037,     11,    220,
         128257,    311,   4148,    279,   4553,  11914,   1603,    420,   4037,
             11,    323,    220, 128258,    311,  21512,   4395,    499,   3077,
           5439,    323,   1212,   1578,     13, 128006,    882, 128007,    271,
           2014,    512,  15029,     38,  94412,  17057,  44895,    323,   4332,
          3956

In [28]:
output = sample.copy()
# output["logits"] = torch.tensor(output["input_ids"]).reshape(1, -1, 1)
output["logits"] = torch.zeros(1, output["input_ids"].shape[-1], len(tokenizer.get_vocab())) - 100
for i in range(output["input_ids"].shape[-1]):
    output["logits"][0, i, output["input_ids"][0, i]] = 100
output["hallucination_logits"] = torch.tensor(output["hallucination_labels"]).reshape(1, -1, 1)

/var/folders/6s/ld6gt9c96lb_b7_r9190nt3w0000gn/T/ipykernel_67397/1449213386.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output["hallucination_logits"] = torch.tensor(output["hallucination_labels"]).reshape(1, -1, 1)


In [29]:
output["logits"][0, :-1, ...] = output["logits"].clone()[0, 1:, ...]

In [30]:
trainer.compute_loss(sample, output, len(tokenizer.get_vocab()))

torch.Size([1, 330, 128259])
torch.Size([1, 330, 1])
torch.Size([1, 330])
torch.Size([1, 330])
tensor([-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    1,
           1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


{'loss': tensor(2.1412e-05),
 'token_loss': tensor(0.),
 'hallucination_loss': tensor(4.2823e-05),
 'f1_score': 1.0}